In [29]:
!pip install dash

In [30]:
!pip install jupyter-dash

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import plotly as plt
import plotly.express as px
import plotly.graph_objects as go

import dash
from dash import dcc
from dash import callback
from dash import html
from dash.dependencies import Input, Output

## Home Page Interactive Globe

In [105]:
import plotly.graph_objects as go
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2014_world_gdp_with_codes.csv')
df['GDP (BILLIONS)'] = np.log(df['GDP (BILLIONS)'])


globe = go.Figure(go.Choropleth(
    z=df['GDP (BILLIONS)'],  # Valores para o mapa de calor
    colorscale='Reds',  # Esquema de cores
    text=df['COUNTRY'],  # Texto que aparecerá quando passar o mouse sobre os países
    locations=df['CODE'],  # Códigos dos países
    zmin=df['GDP (BILLIONS)'].min(),  # Valor mínimo para o mapa de calor
    zmax=df['GDP (BILLIONS)'].max(),  # Valor máximo para o mapa de calor
    colorbar_title='GDP (log)',  # Título da barra de cores
    marker_line_width=1  # Largura da linha do marcador
))

# Configurações de layout
globe.update_layout(
    geo=dict(
        showframe = False,
        showcoastlines = False,
        projection = dict(type = 'orthographic'),
        resolution = 110,
        showcountries = True,
        countrycolor = '#d1d1d1',
        showocean = True,
        oceancolor = '#c9d2e0',
        showlakes = True,
        lakecolor = '#99c0db',
        showrivers = True,
        rivercolor = '#99c0db'
    ),
    width=1200,  # Largura da figura
    height=900  # Altura da figura
)

# Exibindo o gráfico
globe.show()

## North America

## South + Central America

## Africa

## Eastern Europe

## Western Europe

## France

### GFCF

In [6]:
gfcf = pd.read_csv("global/dataglobal/clean_GFCF.csv")
gfcf = gfcf[gfcf["Zone de référence"].isin(['France', 'Colombie', 'Pologne','Nouvelle-Zélande', 'États-Unis','Suède','Union européenne (27 pays à partir du 01/02/2020)', 'Roumanie', 'Hongrie', 'Italie', 'Norvège', 'Bulgarie', 'Espagne','Japon', 'Australie','Corée', 'Canada', 'Allemagne','Suisse','Chili','Grèce'])]
gfcf["Zone de référence"] = np.where(gfcf["Zone de référence"] == 'Union européenne (27 pays à partir du 01/02/2020)','Union européenne',gfcf["Zone de référence"])
gfcf.rename(columns={'Zone de référence':'Countries'},inplace=True)
gfcf.drop(columns=['Unnamed: 0'],inplace=True)
gfcf.dropna(axis=0,inplace=True)
gfcf.head()

,Countries,Secteur institutionnel,Instruments financiers et actifs non financiers,TIME_PERIOD,OBS_VALUE,Multiplicateur d'unité,CURRENCY
117,France,Economie totale,"Actifs fixes, brut",1980-06-01,-0.001915,Millions,EUR
118,France,Economie totale,"Actifs fixes, brut",1980-09-01,-0.001013,Millions,EUR
119,France,Economie totale,"Actifs fixes, brut",1980-12-01,-0.007374,Millions,EUR
120,France,Economie totale,"Actifs fixes, brut",1981-03-01,-0.005284,Millions,EUR
121,France,Economie totale,"Actifs fixes, brut",1981-06-01,0.001727,Millions,EUR


In [7]:
gfcf_plot = px.line(gfcf, x='TIME_PERIOD', y="OBS_VALUE",color='Countries',facet_col="Instruments financiers et actifs non financiers", facet_col_wrap=1)

gfcf_plot.update_layout(xaxis_range=['2000-01-01','2024-04-01'],
                    yaxis_range=[gfcf[gfcf['TIME_PERIOD']>='2000-01-01']['OBS_VALUE'].min()-0.2,gfcf[gfcf['TIME_PERIOD']>='2000-01-01']['OBS_VALUE'].max()+0.2],
                    title_text= "Gross Fixed Capital Formation")

for i in range(len(gfcf_plot.layout.annotations)):
    gfcf_plot.update_xaxes(rangeslider_visible=(i == len(gfcf_plot.layout.annotations)-1), row=i, col=1)

#gfcf_plot.update_xaxes(rangeslider_visible=True)

gfcf_plot.show()

In [138]:
'''import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import dash_core_components as dcc
import dash_html_components as html
from dash import Dash
from dash import callback_context
from dash.exceptions import PreventUpdate
from dash.dependencies import Input, Output, State'''

# Initialize the Dash app
app = dash.Dash(__name__, suppress_callback_exceptions=True, index_string='''<!DOCTYPE html>
<html>
    <head>
        <link rel="preconnect" href="https://fonts.googleapis.com">
        <link rel="preconnect" href="https://fonts.gstatic.com" crossorigin>
        <link href="https://fonts.googleapis.com/css2?family=Angkor&display=swap" rel="stylesheet">
        {%metas%}
        <title>{%title%}</title>
        {%favicon%}
        {%css%}
    </head>
    <body>
        {%app_entry%}
        <footer>
            {%config%}
            {%scripts%}
            {%renderer%}
        </footer>
    </body>
</html>
''')

# Define layout for the home page
def homepage_layout():
    return html.Div(
        children =[
            html.Div(
                className="app-header",
                children=[
                    html.H1(children="Covid Worldwide Analysis", className='app-header--title'),
                    html.P(children="Evolution of the economy before, during and after the Covid pandemic. Continental and global analysis.",
                            className='app-header--description'),
                ]
            ),
            html.Div(
                className="app-nav",
                children=[
                    html.Ul(
                        children=[
                            html.A('Employee Compensation', href='/employee_compensation'),
                            html.A('Employment Rate', href='/employment_rate'),
                            html.A('Final Consumption Expenditure', href='/final_consumption_expenditure'),
                            html.A('GFCF', href='/gfcf'),
                            html.A('GDP', href='/gdp')
                        ]
                    )
                ]
            ),
            html.Div(
                children=[
                    dcc.Graph(figure=globe),
                    #dcc.Graph(figure=studies_level_mean_fig),
                    #dcc.Graph(figure=grouped_studies_level_fig)         
                ],
                style = {"display": "flex",
                    "flex-direction": "row",
                    "justify-content": "center"
                }
            )
        ]
    )


# Define layout for page 1
def page1_layout():
    return html.Div([
        html.H1("Page 2"),
        html.P("This is the content of Page 2."),
        dcc.Link('Go to Home', href='/')
    ])

# Define layout for page 2
def page2_layout():
    return html.Div([
        html.H1("Page 2"),
        html.P("This is the content of Page 2."),
        dcc.Link('Go to Home', href='/')
    ])



# Define layout for page 3
def page3_layout():
    return html.Div([
        html.H1("Page 2"),
        html.P("This is the content of Page 2."),
        dcc.Link('Go to Home', href='/')
    ])

# Define layout for page 4
def page4_layout():
    return html.Div(
        html.Div(
            children=[
                html.Div(
                    className="app-header",
                    children=[
                        html.H1(children="Covid Worldwide Analysis", className='app-header--title'),
                        html.P(
                            children="Evolution of the economy before, during and after the Covid pandemic. Continental and global analysis.",
                            className='app-header--description'),
                    ],
                ),
                html.Div(
                    children=[
                        dcc.Link('Go to Home', href='/'),
                        dcc.Dropdown(
                        gfcf['Countries'].unique(),
                        value=['France'],
                        multi = True,
                        id='countries_filter'),
                    ],
                    style={'width': '100%', 'margin':'07px'}
                    ),
                html.Div(
                    className = 'plot_card',
                    children=[
                        dcc.Graph(id='gfcf_plot'),
                        #dcc.Graph(figure=studies_level_mean_fig),
                        #dcc.Graph(figure=grouped_studies_level_fig)         
                    ],
                )
            ],
            style=wrapper
        )
    )

# Define layout for page 5
def page5_layout():
    return html.Div([
        html.H1("Page 2"),
        html.P("This is the content of Page 2."),
        dcc.Link('Go to Home', href='/')
    ])

@app.callback(
    Output('gfcf_plot', 'figure'),
    Input('countries_filter', 'value'))
def update_graph(selected_categories):

    filtered_gfcf = gfcf[gfcf['Countries'].isin(selected_categories)]

    fig = px.line(filtered_gfcf, x='TIME_PERIOD', y="OBS_VALUE",color='Countries',facet_col="Instruments financiers et actifs non financiers", facet_col_wrap=1)

    
    fig.update_layout(xaxis_range=['2000-01-01','2024-04-01'],
                    yaxis_range=[filtered_gfcf[(filtered_gfcf['TIME_PERIOD']>='2000-01-01')]['OBS_VALUE'].min()-0.05,filtered_gfcf[(filtered_gfcf['TIME_PERIOD']>='2000-01-01')]['OBS_VALUE'].max()+0.05],
                    title_text= "Gross Fixed Capital Formation")

    return fig

# Define callback to display the corresponding layout based on the current pathname
@app.callback(Output('page-content', 'children'),
              [Input('url', 'pathname')])
def display_page(pathname):
    if pathname == '/employee_compensation':
        return page1_layout()
    elif pathname == '/employment_rate':
        return page2_layout()
    elif pathname == '/final_consumption_expenditure':
        return page3_layout()
    elif pathname == '/gfcf':
        return page4_layout()
    elif pathname == '/gdp':
        return page5_layout()
    else:
        return homepage_layout()

# Define the layout of the entire app
app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    html.Div(id='page-content')
])

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

In [72]:
import pandas as pd
from geopy.geocoders import Nominatim
import time



# Supondo que você já tenha o DataFrame 'df' carregado com os nomes dos países

# Inicialize o geocoder
geolocator = Nominatim(user_agent="geoapiExercises")

# Função para obter as coordenadas geográficas de um país
def get_coordinates(country):
    time.sleep(5)  # Esperar 1 segundo entre as solicitações
    location = geolocator.geocode(country)
    if location:
        return location.latitude, location.longitude
    else:
        return None, None

# Aplicar a função aos nomes dos países para obter as coordenadas geográficas
df['LAT'], df['LONG'] = zip(*df['COUNTRY'].apply(get_coordinates))

# Agora você pode usar as colunas LAT e LONG para plotar os países em um globo 3D

GeocoderInsufficientPrivileges: Non-successful status code 403

In [64]:
df

,COUNTRY,GDP (BILLIONS),CODE
0,Afghanistan,3.077773,AFG
1,Albania,2.595255,ALB
2,Algeria,5.428468,DZA
3,American Samoa,-0.287682,ASM
4,Andorra,1.568616,AND
...,...,...,...
217,Virgin Islands,1.625311,VGB
218,West Bank,1.893112,WBG
219,Yemen,3.816613,YEM
220,Zambia,3.242983,ZMB


### Final Consumption Expenditure

## Oceania

## Southeast Asia

## Asia

## China + India